Limpieza de datos Rooms

In [3]:
import pandas as pd
import numpy as np
import re 

In [4]:
data_location = "./Data/properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [161]:
data["rooms_completo"] =data["rooms"]
data_room_null_description = data.description.astype(str).str.lower()
data_room_null_title = data.title.astype(str).str.lower()
display(data_room_null_description)
display(data_room_null_title)

0         2 ambientes tipo casa planta baja por pasillo,...
1         venta de departamento en décimo piso al frente...
2         2 ambientes 3er piso lateral living comedor am...
3         ph 3 ambientes con patio. hay 3 deptos en lote...
4         departamento con fantástica iluminación natura...
                                ...                        
121215    torre forum alcorta - máxima categoría.impecab...
121216    excelente e impecable casa en venta en las lom...
121217    venta departamento ambiente divisible a estren...
121218    2 amb al contrafrente,  luminoso. el departame...
121219    departamento de 77 m2 apto profesional. edific...
Name: description, Length: 121220, dtype: object

0                        2 amb tipo casa sin expensas en pb
1         venta depto 2 dorm. a estrenar 7 e/ 36 y 37   ...
2                  2 amb 3er piso con ascensor apto credito
3                              ph 3 amb. cfte. reciclado   
4         depto 2 amb al contrafrente zona centro/plaza ...
                                ...                        
121215           torre forum alcorta- impecable 3 ambientes
121216    ruca inmuebles | venta | lomas de san isidro |...
121217    venta departamento ambiente divisible a estren...
121218    2 amb. c/ dep. de servicio al contrafrente| re...
121219           departamento apto profesional en el centro
Name: title, Length: 121220, dtype: object

In [53]:
print("Primer grupo de filas sin rooms")
data["rooms_completo"].isnull().value_counts()

Primer grupo de filas sin rooms


True     73830
False    47390
Name: rooms_completo, dtype: int64

Seteamos los que son monoambientes
Primero buscamos las palabras que identifiquen al monoambiente. 
Luego buscamos las coincidencias dentro del universo de coincidencias del campo descripción
donde hubo coincidencias, le asignamos 1. 

In [128]:
def fun_buscar_patt(data_validar,patt_room):
    patt_regex = re.compile(patt_room)
    data_match = data_validar.apply(lambda x:  x if x is np.NaN else patt_regex.search(x))
    return data_match

In [129]:
def fun_buscar_disminuir(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes tipo:",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [142]:
def fun_buscar_disminuir_mapper(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
       'un': 1,'uun':1, 'uno': 1,'dos': 2,'tres': 3,'cuatro':4,
        'cinco':5,'sinco':5,'seis':6,'ceis':6,'siete':7,
        'ciete':7,'ocho':8,'nueve':9,'diez':10,'once':11,
        'doce':12,'trece':13,'siete':14,'quince':15,
        'dieciseis':16,'diecisiete':17,'dieciocho':18,'diecinueve':19,
        'veinte':20
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [160]:
def fun_buscar_disminuir_mapper_2(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
                'segundo': 2,'tercer':3, 'cuarto': 4,'quinto': 5,'sexto': 6
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [162]:
data_room_match = fun_buscar_patt(data_room_null_description,r"(mono ambiente|monoambiente|mono-ambiente)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

Primera disminución Monoambientes


True     69080
False    52140
Name: rooms_completo, dtype: int64

In [163]:
resultado = fun_buscar_disminuir(data_room_null_description,r"(\d+)[\s]?habitaci",0)
print(resultado)

Disminución en función a los ambientes tipo: (\d+)[\s]?habitaci
True     67494
False    53726
Name: rooms_completo, dtype: int64


In [164]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+)[\s]?(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(dormitorio|domitorio)
False    65631
True     55589
Name: rooms_completo, dtype: int64


In [165]:
resultado = fun_buscar_disminuir_mapper_2(data_room_null_description,r"(\w+)[\s]?(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(dormitorio|domitorio)
False    66319
True     54901
Name: rooms_completo, dtype: int64


In [166]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+)[\s]?(ambiente)",1)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(ambiente)
False    68212
True     53008
Name: rooms_completo, dtype: int64


In [167]:
resultado = fun_buscar_disminuir(data_room_null_description,r"(\d+)[\s]?(amb)",1)
print(resultado)

Disminución en función a los ambientes tipo: (\d+)[\s]?(amb)
False    84113
True     37107
Name: rooms_completo, dtype: int64


In [168]:
resultado = fun_buscar_disminuir(data_room_null_description,r"(\d+)[\s]?(dorm)",0)
print(resultado)

Disminución en función a los ambientes tipo: (\d+)[\s]?(dorm)
False    102522
True      18698
Name: rooms_completo, dtype: int64


In [169]:
resultado = fun_buscar_disminuir(data_room_null_description,r"(\d+)[\s]?(suite)",0)
print(resultado)

Disminución en función a los ambientes tipo: (\d+)[\s]?(suite)
False    102637
True      18583
Name: rooms_completo, dtype: int64


Buscamos ahora las restantes en el título

In [170]:
resultado = fun_buscar_disminuir(data_room_null_title,r"(\d)[\s]?amb",1)
print(resultado)

Disminución en función a los ambientes tipo: (\d)[\s]?amb
False    103577
True      17643
Name: rooms_completo, dtype: int64


In [171]:
resultado = fun_buscar_disminuir_mapper(data_room_null_title,r"(\w+)[\s]?(amb)",1)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(amb)
False    103734
True      17486
Name: rooms_completo, dtype: int64


## `SETEANDO RESTO DE ROOMS EN NULL`
AQUÍ SETEO EL RESTO QUE SE ENCUENTRA EN NULL

In [154]:
print(data["rooms_completo"].isnull().value_counts())
data_mask = data["rooms_completo"].isnull()
data_encontrada = data_room_null_description.apply(lambda x: x if x is np.NaN else x.count("dormitorio|habitacion"))
data.loc[data_mask,"rooms_completo"] = data_encontrada[data_mask]
print(data["rooms_completo"].isnull().value_counts())

data_encontrada.max()
#data["rooms_completo"].isnull().value_counts()
# encontradas_mask = encontradas.isnull()
# encontradas[encontradas_mask]
# encontradas

False    103392
True      17828
Name: rooms_completo, dtype: int64
False    121220
Name: rooms_completo, dtype: int64


0

## `AGREGANDO CAMPOS NUEVOS`
Se crean nuevos campos para definir características que pueden revalorizar la propiedad
____

In [155]:
data_encontrada1= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(pileta|piscina|pisina|picina)")
encontradas_pileta = data_encontrada1[data_encontrada1.notnull()].apply(lambda x: x.group(0))
data["PILETA"] = encontradas_pileta.apply(lambda x: False if x.count("sin")>0 else True)
data["PILETA"].fillna(value=False, inplace=True)
data["PILETA"].value_counts()

False    104706
True      16514
Name: PILETA, dtype: int64

In [156]:
data_encontrada_parrilla= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(parrilla|asado|grill)")
encontradas_parrilla = data_encontrada_parrilla[data_encontrada_parrilla.notnull()].apply(lambda x: x.group(0))
data["PARRILLA"] = encontradas_parrilla.apply(lambda x: False if x.count("sin")>0 else True)
data["PARRILLA"].fillna(value=False, inplace=True)
# data["PARRILLA"]= data_encontrada_parrilla.notnull()
data["PARRILLA"].value_counts()

False    93320
True     27900
Name: PARRILLA, dtype: int64

In [157]:
data_encontrada_ascensor= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(ascensor|elevador|acensor|asensor)")
encontradas_ascensor = data_encontrada_ascensor[data_encontrada_ascensor.notnull()].apply(lambda x: x.group(0))
data["ASCENSOR"] = encontradas_ascensor.apply(lambda x: False if x.count("sin")>0 else True)
data["ASCENSOR"].fillna(value=False, inplace=True)
data["ASCENSOR"].value_counts()
#ME PARECE QUE CON TAN POCO DATO NO VALE LA PENA CREAR ESTA COLUMNA.

False    116631
True       4589
Name: ASCENSOR, dtype: int64

In [158]:
data_encontrada_patio= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(patio|jardin)")
encontradas_patio = data_encontrada_patio[data_encontrada_patio.notnull()].apply(lambda x: x.group(0))
data["PATIO_JARDIN"] = encontradas_patio.apply(lambda x: False if x.count("sin")>0 else True)
data["PATIO_JARDIN"].fillna(value=False, inplace=True)
data["PATIO_JARDIN"].value_counts()

False    104289
True      16931
Name: PATIO_JARDIN, dtype: int64

ESTO ES PARA REALIZAR VALIDACIÓN DE DATOS


______


In [159]:
data_room_search = data[["property_type","description"]][data["rooms_completo"].isnull()]
data_room_search.sample(8)

ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
#76874,100557,35002 = 2
#46551 = 1
#48411 = 4
#66284,75074 = cant, palabra dormitorio
idreg= 1507
print(data.title[idreg])
display(data.loc[idreg:idreg,["property_type","rooms_completo","description"]])
display(data_room_null_description[idreg])
data_finded= fun_buscar_patt(data["description"][data["rooms_completo"].isnull()],"r(dormitorio)")
print("--___----")
display(data_finded[data_finded.notnull()])

In [ ]:
data.loc[idreg:idreg+1,:]

In [ ]:
data.property_type.value_counts()

In [ ]:
print(resultado[resultado.notnull()].value_counts())

In [ ]:
#display(data.loc[:,["property_type","rooms_completo","description"]])
#data.pivot_table('survived', index='sex', columns='class', margins=True, margins_name= "totales")
#resultado = data.pivot_table(["rooms","rooms_completo"],index='rooms_completo', columns="property_type", margins=True)
vale = (data.loc[:,"rooms_completo"]) > 10
#data.groupby('BARRIO')['M2'].median().sort_values(ascending=False)
data.loc[vale,"rooms_completo"].sort_values(ascending=False)

In [ ]:

# aggfunc={'PRECIOTEXTM':[np.mean,np.std,len],
#                           'AMBIENTES': [np.mean,np.std]})

resultado = data[["property_type","rooms","rooms_completo"]]
resultado.groupby("property_type")["rooms"].mean()